In [16]:
import pandas as pd
import numpy as np

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

from sklearn import svm
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score, precision_score, classification_report, recall_score
import gensim.downloader as api
from gensim.models.doc2vec import Doc2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [2]:
train_data = pd.read_csv('data/entailment_train.csv')
test_data = pd.read_csv('data/entailment_test.csv')

In [14]:
train_sent1 = train_data['sent1']
train_sent2 = train_data['sent2']
train_label = train_data['label']

test_sent1 = test_data['sent1']
test_sent2 = test_data['sent2']
test_label = test_data['label']

In [12]:
def mean_vectorizor(model, tokenized_sent, dim):
    
    return np.array([
            np.mean([model[w] for w in words if w in model.vocab]
                    or [np.zeros(dim)], axis=0)
            for words in tokenized_sent
        ])

def google_news_word2vec(train_event_1, train_event_2, test_event_1, test_event_2):
    
    model = KeyedVectors.load_word2vec_format('../../word_embedings/GoogleNews-vectors-negative300.bin', binary=True)
    
    train_vec1 = mean_vectorizor(model, train_event_1, 300)
    train_vec2 = mean_vectorizor(model, train_event_2, 300)
    test_vec1 = mean_vectorizor(model, test_event_1, 300)
    test_vec2 = mean_vectorizor(model, test_event_2, 300)
    
    return train_vec1, train_vec2, test_vec1, test_vec2, model

In [13]:
train_vec1, train_vec2, test_vec1, test_vec2, model = google_news_word2vec(train_sent1, train_sent2, test_sent1, test_sent2)

In [17]:
train_vec = np.concatenate((train_vec1, train_vec2), axis=1)
test_vec = np.concatenate((test_vec1, test_vec2), axis=1)

print(train_vec.shape)
print(test_vec.shape)

clf = svm.SVC(kernel = 'linear').fit(train_vec, train_label)
#clf = KNeighborsClassifier(n_neighbors=3).fit(train_vec, train_label)
#clf = MultinomialNB().fit(train_vec, train_label)
pred_labels = clf.predict(test_vec)
print("confusion_matrix: \n", confusion_matrix(test_label, pred_labels))
print("Accuracy : ",accuracy_score(test_label, pred_labels))
print("Classification Report: \n", classification_report(test_label, pred_labels))

(20228, 600)
(5057, 600)
confusion_matrix: 
 [[1166 1223]
 [ 530 2138]]
Accuracy :  0.6533517895985762
Classification Report: 
               precision    recall  f1-score   support

           0       0.69      0.49      0.57      2389
           1       0.64      0.80      0.71      2668

    accuracy                           0.65      5057
   macro avg       0.66      0.64      0.64      5057
weighted avg       0.66      0.65      0.64      5057

